## 💿 Requirements

The code is built based on Python 3.9, PyTorch 1.10.0, and [EasyTorch](https://github.com/cnstark/easytorch).
You can install PyTorch following the instruction in [PyTorch](https://pytorch.org/get-started/locally/). For example:

```bash
pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
```

After ensuring that PyTorch is installed correctly, you can install other dependencies via:

```bash
pip install -r requirements.txt
```

In [ ]:
!pip install -r requirements.txt


# Data Preparation

## Download Raw Data

You can download all the raw datasets at [Google Drive](https://drive.google.com/file/d/1PY7IZ3SchpyXfNIXs71A2GEV29W5QCv2/view?usp=sharing) or [Baidu Yun](https://pan.baidu.com/s/1CXLxeHxHIMWLy3IKGFUq8g?pwd=blf8), and unzip them to `datasets/raw_data/`.

## Pre-process Data

You can pre-process all data via:

```bash
cd /path/to/your/project
bash scripts/data_preparation/all.sh
```

Then the `dataset` directory will look like this:

```text
datasets
   ├─METR-LA
   ├─METR-BAY
   ├─PEMS04
   ├─raw_data
   |    ├─PEMS04
   |    ├─PEMS-BAY
   |    ├─METR-LA
   ├─README.md
```


In [1]:
!python scripts/data_preparation/METR-LA/generate_training_data.py --history_seq_len 12
!python scripts/data_preparation/METR-LA/generate_training_data.py --history_seq_len 2016
!python scripts/data_preparation/PEMS-BAY/generate_training_data.py --history_seq_len 12
!python scripts/data_preparation/PEMS-BAY/generate_training_data.py --history_seq_len 2016
!python scripts/data_preparation/PEMS04/generate_training_data.py --history_seq_len 12
!python scripts/data_preparation/PEMS04/generate_training_data.py --history_seq_len 4032

----------------------------------------------------------------------
|      data_file_path = datasets/raw_data/METR-LA/METR-LA.h5         |
|                 dow = True                                         |
|      future_seq_len = 12                                           |
|     graph_file_path = datasets/raw_data/METR-LA/adj_METR-LA.pkl    |
|     history_seq_len = 12                                           |
|          output_dir = datasets/METR-LA                             |
|      target_channel = [0]                                          |
|                 tod = True                                         |
|         train_ratio = 0.7                                          |
|         valid_ratio = 0.1                                          |
----------------------------------------------------------------------
raw time series shape: (34272, 207, 1)
number of training samples:23974
number of validation samples:3425
number of test samples:6850
mean (training

## <span id="jump"> 🎯 Train STEP based on a Pre-trained TSFormer </span>

```bash
python step/run.py --cfg='step/STEP_$DATASET.py' --gpus='0, 1'
# python step/run.py --cfg='step/STEP_METR-LA.py' --gpus='0, 1'
# python step/run.py --cfg='step/STEP_PEMS-BAY.py' --gpus='0, 1'
# python step/run.py --cfg='step/STEP_PEMS04.py' --gpus='0, 1'
```

Replace `$DATASET_NAME` with one of `METR-LA`, `PEMS-BAY`, `PEMS04` as shown in the code above. 
Configuration file `step/STEP_$DATASET.py` describes the forecasting configurations.
Edit `BATCH_SIZE` and `GPU_NUM` in the configuration file and `--gpu` in the command line to run on your own hardware.
Note that different GPU number leads to different real batch sizes, affecting the learning rate setting and the forecasting accuracy.

Our training logs are shown in [training_logs/STEP_METR-LA.log](./training_logs/STEP_METR-LA.log), [training_logs/STEP_METR-LA.log](./training_logs/STEP_METR-LA.log) and [training_logs/STEP_PEMS-BAY.log](./training_logs/STEP_PEMS-BAY.log).


In [2]:
import torch
 
print(torch.cuda.is_available())
#cuda是否可用；
 
print(torch.cuda.device_count())
#返回gpu数量；
 
print(torch.cuda.get_device_name(0))
#返回gpu名字，设备索引默认从0开始；
 
print(torch.cuda.current_device())
#返回当前设备索引；
!nvidia-smi

D:\python-venv\STEP\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False
0


AssertionError: Torch not compiled with CUDA enabled

In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_May__3_19:41:42_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.3, V11.3.109
Build cuda_11.3.r11.3/compiler.29920130_0


In [3]:
!python step/run.py --cfg='step/STEP_METR-LA.py' --gpus='1'

2022-12-10 16:03:53,493 - easytorch-launcher - INFO - Launching EasyTorch training.
Traceback (most recent call last):
  File "D:\python-venv\STEP\step\run.py", line 27, in <module>
    launch_training(args.cfg, args.gpus)
  File "D:\python-venv\STEP\basicts\launcher.py", line 19, in launch_training
    easytorch.launch_training(cfg=cfg, gpus=gpus, node_rank=node_rank)
  File "C:\Users\15158\.conda\envs\STEP\lib\site-packages\easytorch\launcher\launcher.py", line 80, in launch_training
    cfg = init_cfg(cfg, node_rank == 0)
  File "C:\Users\15158\.conda\envs\STEP\lib\site-packages\easytorch\launcher\launcher.py", line 13, in init_cfg
    cfg = import_config(cfg, verbose=save)
  File "C:\Users\15158\.conda\envs\STEP\lib\site-packages\easytorch\config.py", line 245, in import_config
    cfg = __import__(path, fromlist=[cfg_name]).CFG
ModuleNotFoundError: No module named "'step"
